In [5]:
import pandas as pd
import os
import glob
import numpy as np

# setting notebook
pd.set_option('display.max_columns', None)

In [7]:
path = "../../data/raw/00-po-all"

all_files = glob.glob(os.path.join(path, "*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, dtype=str ,index_col=None, header=10)
    li.append(df)

po_cbn= pd.concat(li, axis=0, ignore_index=True)

In [8]:
#Drop null rows
po_nonull = po_cbn.dropna(subset=['Số đơn hàng\n(Order No)']).reset_index(drop=True)

In [9]:
#Rename columns
po_rename = po_nonull.rename(columns={'Unnamed: 24': 'Chứng từ KH ký nhận', 'Unnamed: 26': 'Chứng từ khác'}) 

In [10]:
#Convert datetime
date_cols = ['Ngày về\n(Receiving Date)',
              'Hạn chót giao hàng\n(Delivery Deadline)',
              'Ngày thực tế giao hàng']
po_rename[date_cols] = po_rename[date_cols].apply(pd.to_datetime, format='%d/%m/%Y')

po_rename['Chứng từ KH ký nhận'] = np.where(~po_rename['Chứng từ KH ký nhận'].astype(str).str.contains(r'^\d{2}\/\d{2}\/\d{4}', regex=True, na=False), pd.NaT, po_rename['Chứng từ KH ký nhận'])
po_rename['Chứng từ KH ký nhận'] = po_rename['Chứng từ KH ký nhận'].apply(pd.to_datetime, format='%d/%m/%Y')



In [11]:
po_joined = po_rename.copy()